In [1]:
import operator
import time
import copy
import sys
import gc
import json
import os
import re
import sys
import pickle
import base64
import nltk   
import json

In [2]:
print(os.listdir("../input/search-engine"))

['title_output.txt', 'title_position.pickle', 'category_output.txt', 'body_text_output.txt', 'titles.txt', 'infobox_output.txt']


In [3]:
word_position = json.load(open("../input/word-position-1/word_position.json", "r"))

In [4]:
def read_file(testfile):
    with open(testfile, 'r') as file:
        queries = file.readlines()
    return queries

In [5]:
def write_file(outputs, path_to_output):
    '''outputs should be a list of lists.
        len(outputs) = number of queries
        Each element in outputs should be a list of titles corresponding to a particular query.'''
    with open(path_to_output, 'w') as file:
        for output in outputs:
            for line in output:
                file.write(line.strip() + '\n')
            file.write('\n')

In [6]:
def mapping_shortform(field) :
    
    field = field.lower()
    
    if field == "title" :
        return "t"
    elif field == "infobox" :
        return "i"
    elif field == "category" :
        return "c"
    elif field == "body" :
        return "d"
    elif field == "ref" :
        return "d"
    else :
        return field

In [7]:
def search(path_to_index, queries):
    
    global word_position
    stemmer = nltk.stem.SnowballStemmer('english')

    stop_words = {}
    reg = re.compile("\"|,| ")
    stop_file = open(path_to_index+"/stop-words/stop_words.txt", "r")
    content = stop_file.read()
    content = re.split(reg, content)
    for word in content :
        if word :
            stop_words[word] = True
        
    title_tags = open(path_to_index+"/search-engine/titles.txt", "r")
    title_position = pickle.load(open(path_to_index+"/search-engine/title_position.pickle", "rb"))
#     word_position = json.load(open(path_to_index+"/word-position-1/word_position.json", "r"))

    field_map = {"t" : 0, "d" : 1, "i" : 2, "c" : 3}
    field_chars = ["title_output", "body_text_output", "infobox_output", "category_output"] 
    files = []

    for f in field_chars :
        file = path_to_index+ "/search-engine/" + f + ".txt"
        fp = open(file, "r")
        files.append(fp)
        
    final_result = []
    for query in queries :
        
        print(query)

        result = []
        documents = dict()
        query_words = list()

        # query = query.lower().strip()
#         start = time.time()
#         if (query == "exit") :
#             break

        if ":" in query :
            query_bag = query.split(" ")
            t_result=list()
            flag2=0
            for q in query_bag :
                field_query = q.split(":")
                field = field_query[0]
                query = field_query[1]
                field = mapping_shortform(field)
                query_words = query.split()
                for word in query_words :
                    print(word)
                    word = stemmer.stem(word)
                    print("stem",word)
                    if word in word_position and field in word_position[word] :
                        position = word_position[word][field]
                        files[field_map[field]].seek(position)
                        intersection=list()
                        s = files[field_map[field]].readline()[:-1] # remove "/n" [:-1] & read full line of posting list
                        print("posting List : ",s)
                        if "," in s :
                            items = s.split(",")
                            for item in items :
                                document_score = item.split(":")
                                doc_id = document_score[0]
                                score = document_score[1]
                                tt = 1
                                print(type(doc_id))
                                print(doc_id," : ",score)
                                if doc_id in documents :
                                    documents[doc_id] = documents[doc_id] + float(score)
                                else :
                                    documents[doc_id] = float(score)
                        else :
                            document_score = item.split(":")
                            doc_id = document_score[0]
                            score = document_score[1]
                            tt = 1
                            union_list = list()
                            if doc_id in documents :
                                documents[doc_id] = documents[doc_id] + float(score)
                            else :
                                documents[doc_id] = float(score)
                        

        else :    
            query_bag = query.split()      
            length = len(query_bag)
            for i in range(length) :
                query_bag[i] = stemmer.stem(query_bag[i])
                
            for word in query_bag :
                if word not in stop_words and word in word_position:
                    query_words.append(word)

            for word in query_words :
                docs = list()
                flag2=0
                positions = word_position[word]
                for field in positions.keys() :
                    position = positions[field]
                    intersection=list()
                    files[field_map[field]].seek(position)
                    s = files[field_map[field]].readline()[: -1]
                    if "," in s :
                        items = s.split(",")
                        for item in items :
                            document_score = item.split(":")
                            doc_id = document_score[0]
                            score = document_score[1]
                            tt = 1
                            if doc_id in documents :
                                documents[doc_id] =  documents[doc_id] + float(score)
                            else :
                                documents[doc_id] = float(score)
                                
                    else :
                        document_score = item.split(":")
                        doc_id = document_score[0]
                        score = document_score[1]
                        tt = 1
                        union_list = list()
                        if doc_id in documents :
                            documents[doc_id] =  documents[doc_id] + float(score)
                        else:
                            documents[doc_id] = float(score)
        
        documents = sorted(documents.items(), key = operator.itemgetter(1), reverse = True)
        count = 1
        for document in documents :
            
            position = title_position[int(document[0]) - 1]
            title_tags.seek(position)
            title = title_tags.readline()[: -1]
            print(title,document[1])
            result.append(title)
            count += 1
#             if count > 10 :
#                     break
                    
        final_result.append(result)

    print(final_result)
    return final_result

In [8]:
def main():
#     path_to_index = sys.argv[1]
#     testfile = sys.argv[2]
#     path_to_output = sys.argv[3]

    path_to_index = "../input"
    testfile = "../input"
    path_to_output = "../input"

    queries = ["title:gandhi body:arjun infobox:gandhi category:gandhi ref:gandhi"]
#     queries = read_file(testfile)
    outputs = search(path_to_index, queries)
    write_file(outputs, path_to_output)

In [9]:
if __name__ == '__main__':
    main()

title:gandhi body:arjun infobox:gandhi category:gandhi ref:gandhi
gandhi
stem gandhi
posting List :  11124028:5.49,12105898:5.49,13804399:5.49,13957777:5.49,14459979:5.49,10020413:4.22,10028269:4.22,10048983:4.22,10075685:4.22,10106198:4.22
<class 'str'>
11124028  :  5.49
<class 'str'>
12105898  :  5.49
<class 'str'>
13804399  :  5.49
<class 'str'>
13957777  :  5.49
<class 'str'>
14459979  :  5.49
<class 'str'>
10020413  :  4.22
<class 'str'>
10028269  :  4.22
<class 'str'>
10048983  :  4.22
<class 'str'>
10075685  :  4.22
<class 'str'>
10106198  :  4.22
arjun
stem arjun
posting List :  16028462:11.72,405933:11.54,13669631:11.35,16778229:11.0,14254017:10.61,573812:10.5,1747622:10.4,12073330:10.31,6808806:10.31,6441619:9.97
<class 'str'>
16028462  :  11.72
<class 'str'>
405933  :  11.54
<class 'str'>
13669631  :  11.35
<class 'str'>
16778229  :  11.0
<class 'str'>
14254017  :  10.61
<class 'str'>
573812  :  10.5
<class 'str'>
1747622  :  10.4
<class 'str'>
12073330  :  10.31
<class 'str

IsADirectoryError: [Errno 21] Is a directory: '../input'